# archive to folder

In [ ]:
# nuclio: ignore
import nuclio

In [ ]:
import os
import zipfile
import urllib.request
import tarfile
import json

from mlrun.execution import MLClientCtx
from mlrun.datastore import DataItem

from typing import Union

def open_archive(
    context: MLClientCtx, 
    archive_url: DataItem = "",
    subdir: str = "content",
    key: str = "content"
):
    """Open a file/object archive into a target directory
    
    Currently supports zip and tar.gz
    
    :param context:      function execution context
    :param archive_url:  url of archive file
    :param subdir:       path within artifact store where extracted files
                         are stored
    :param key:          key of archive contents in artifact store
    """
    target_dir = os.path.join(context.artifact_path, subdir)
    os.makedirs(target_dir, exist_ok=True)
    
    archive_url = str(archive_url)
    if archive_url.endswith("gz"):
        with tarfile.open(archive_url, mode="r|gz") as ref:
            ref.extractall(target_dir)
    elif archive_url.endswith("zip"):
        with zipfile.ZipFile(archive_url, "r") as ref:
            ref.extractall(target_dir)
    else:
        raise ValueError(f'unsupported archive type in {archive_url}')
    
    context.log_artifact(key, local_path=subdir)

In [ ]:
# nuclio: end-code

### save

In [ ]:
from mlrun import code_to_function 
# create job function object from notebook code
fn = code_to_function("open_archive", kind="job", with_doc=True,
                      handler=open_archive, image="mlrun/ml-base")

# add metadata (for templates and reuse)
fn.spec.default_handler = "open_archive"
fn.spec.description = "Open a file/object archive into a target directory"
fn.metadata.categories = ["fileutils", "retrieve"]
fn.metadata.labels = {"author": "yaronh"}

fn.save()
fn.export("function.yaml")

## tests

In [ ]:
from mlrun import import_function, mount_v3io, NewTask, run_local

In [ ]:
func = import_function("function.yaml").apply(mount_v3io())

### tar

In [ ]:
run_local(handler=open_archive, inputs={'archive_url': "https://fpsignals-public.s3.amazonaws.com/catsndogs.tar.gz"})

### zip

In [ ]:
run_local(handler=open_archive, inputs={'archive_url': 'http://iguazio-sample-data.s3.amazonaws.com/catsndogs.zip'})